In [1]:
# specify to reload any packages that are changed
# helps with not having to restart the kernel each time
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys
import copy
import corner 
import jax
import os
import scarlet
print(scarlet.__file__)
import astropy.io.fits as fits
from astropy.wcs import WCS
from scarlet.display import AsinhMapping,AsinhPercentileNorm,show_scarlet2_scene,LinearPercentileNorm
import glob
import sep
import tarfile
import matplotlib
import matplotlib.pyplot as plt
import numpyro.distributions as dist
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.wcs import WCS
from scarlet.source import StaticSource,MultiExtendedSource, StaticMultiExtendedSource
import jax.numpy as jnp
import equinox as eqx
from jax import random, jit
import distrax
import optax
from tqdm.auto import tqdm
import pandas as pd
from numpyro.distributions import constraints
import cmasher as cmr
from scarlet2 import *
import scarlet2
from skimage import draw
print(scarlet2.__file__)
# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='gray', interpolation='none', origin='lower')

/home/cw1074/HSC/pulsars/scarlet/scarlet/__init__.py


/home/cw1074/.conda/envs/scarlet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'scarlet2'

In [ ]:
def set_plot_style(doit=True,serif=True,use_tex=True):
    from matplotlib import cycler
    new_rcparams = {
        # Set color cycle: blue, green, yellow, red, violet, gray
        #'#axes.prop_cycle' : cycler('color', ['0C5DA5', '00B945', 'FF9500', 'FF2C00', '845B97', '474747', '9e9e9e']),
        # Set color cycle: blue, orange, green, red, violet, gray
        'axes.prop_cycle' : cycler('color', ['0C5DA5', 'FF9500', '00B945', 'FF2C00', '845B97', '474747', '9e9e9e']),
        # Set default figure size
        'figure.figsize' : (4*1.5, 3*1.5),
        # Set x axis
        'xtick.direction' : 'in',
        'xtick.major.size' : 3*2,
        'xtick.major.width' : 0.5,
        'xtick.minor.size' : 1.5*2,
        'xtick.minor.width' : 0.5,
        'xtick.minor.visible' : True,
        'xtick.top' : True,
        # Set y axis
        'ytick.direction' : 'in',
        'ytick.major.size' : 3*2,
        'ytick.major.width' : 0.5,
        'ytick.minor.size' : 1.5*2,
        'ytick.minor.width' : 0.5,
        'ytick.minor.visible' : True,
        'ytick.right' : True,
        # Set line widths
        'axes.linewidth' : 0.5,
        'grid.linewidth' : 0.5,
        'lines.linewidth' : 1.,
        # Remove legend' frame
        'legend.frameon' : True,
        # Always save as 'tight'
        'savefig.bbox' : 'tight',
        'savefig.pad_inches' : 0.05,
    }
    if use_tex:
        new_rcparams.update({
            # Use LaTeX for math formatting
            'text.usetex' : True,
            'text.latex.preamble' : r'\usepackage{amsmath}'
            })
    if serif:
        new_rcparams.update({
            # Use serif fonts
            'font.serif' : 'Times New Roman',
            'font.family' : 'serif',
            'mathtext.fontset' : 'dejavuserif',
            'legend.fontsize' : 'large',
            'xtick.labelsize' : 'x-large',
            'ytick.labelsize' : 'x-large',
            'axes.labelsize' : 'xx-large',
            })
    if doit:
        plt.rcParams.update(new_rcparams)
    else:
        return new_rcparams
set_plot_style()
plt.rcParams["font.family"] = "Serif"

Provide name and position of source, desired bands, and location of image data.

In [ ]:
getphotometry = True
getgalaxySED = False
getoffset = True
ra = 149.72718
dec = 2.02325
bandall=['g','i','r']
SNR = 5
shifts = [0.0,0.05,0.2,0.5,0.75,1.0,1.5]
pixscale=5
shiftmultiple=5

for shift in shifts[4:5]:
    galSNratio = 2.0
    srcname = 'SNoff'+str(shift)+'_'+str(SNR)+'_'+str(galSNratio)
    bandall=['g','r','i']
src = srcname
datadir='/scratch/gpfs/cw1074/HSCdata/multiressingleshift_priorimage'
imagedir = datadir+'/'+src
if not os.path.isdir(imagedir):
    os.mkdir(imagedir)
plotdir = datadir+'/'+src+'/plots'
print(plotdir)
if not os.path.isdir(plotdir):
    os.mkdir(plotdir)

coord_transient = SkyCoord(ra*u.deg, dec*u.deg, frame='icrs')


Define a function which stacks the images and performs source detection on the stack

In [ ]:
def makeCatalog(observations, lvl=3, wave=True):
    normed_images = np.asarray([obs.data for obs in observations])
    interps = [scarlet.interpolation.interpolate_observation(obs, observations[0]) for obs in observations]
    interps = np.asarray(interps/np.sum(interps))
    detect_image = np.sum(interps,axis=(0,1))
    # Wavelet transform
    wave_detect = scarlet.Starlet.from_image(detect_image).coefficients

    if wave:
        # Creates detection from the first 3 wavelet levels
        detect = wave_detect[:lvl,:,:].sum(axis = 0)
    else:
        detect = detect_image

        # Runs SEP detection
    bkg = sep.Background(detect)
    catalog = sep.extract(detect-bkg.globalback, 3, err=bkg.globalrms)
    background=[]
    bg_rms=[]
    for obs in observations:
        img = obs.data
        if np.size(img.shape) == 3:
            bg_rms.append(np.array([sep.Background(band).globalrms for band in img]))
            background.append(np.array([sep.Background(band).globalback for band in img]))
        else:
            bg_rms.append(sep.Background(img).globalrms)
            background.append(sep.Background(img).globalback)
    return catalog, bg_rms, detect_image, background

For each band, obtain the science, weight and PSF images. Loop over each image and create scarlet1 and scarlet2 observation objects.

In [ ]:
observations=[]
observations_sc2_old=[]
channels=[]
channels_sc2 =[]
channels_zeroed=[]
times=[]
zps =[]
norms=[]
for band in bandall:
    imageout=[]
    psfs=[]
    images = glob.glob(imagedir+'/'+srcname+'_lsst'+band+'_*highres*[0-9].fits')#glob.glob(imagedir+'/warps-HSC-'+band+'-*-pdr3_dud*/warp-*.fits')
    print(imagedir+'/'+srcname+'_lsst'+band+'_*highres*[0-9].fits')
    for i,fn in enumerate(images):
        matches = [images[i].rstrip('.fits')+'_epsf.fits']#glob.glob(imagedir+'/SN-lsst'+band+'*_epsf.fits')
        if len(matches)>0:
                imageout.append(fn)
                psfs.append(matches[0])

        for img,psf in zip(imageout,psfs):
            obs_hdu = fits.open(img)
            data_hsc = obs_hdu[0].data.byteswap().newbyteorder()
            N1, N2 = data_hsc.shape
            data_hsc = data_hsc.reshape(1, N1, N2)

    for ind,(img,psf) in enumerate(zip(imageout,psfs)):
        obs_hdu = fits.open(img)
        try:
            mjd = Time(img.strip('.fits').split('_')[-1],format='mjd').mjd#Time(obs_hdu[0].header['DATE-AVG'],format='isot',scale='utc').mjd
        except KeyError:
            print('No time')
            continue
        data_hsc = obs_hdu[0].data.byteswap().newbyteorder()
        N1, N2 = data_hsc.shape
        data_hsc = data_hsc.reshape(1, N1, N2)
        weight_hsc=np.ones(data_hsc.shape)
        wcs_hsc = WCS(obs_hdu[0].header)
        channel = [band+str(ind)]
        channel_sc2 = (band, str(ind)) 
        channels.append(band+str(ind))
        channels_sc2.append(channel_sc2)
        if mjd>56160:
            channels_zeroed.append(channel_sc2)
        psf_hsc0 = fits.open(psf)[0].data
        Np1, Np2 = psf_hsc0.shape
        p1,p2=psf_hsc0.shape
        psf_hsc = scarlet.ImagePSF(psf_hsc0)
        r=1
        
        obs_hsc = scarlet.Observation(data_hsc,
                wcs=wcs_hsc,
                psf=psf_hsc,
                channels=channel,
                weights=weight_hsc)
        observations.append(obs_hsc)
        times.append(mjd)
        obs_sc2 = scarlet2.Observation(jnp.asarray(data_hsc), jnp.asarray(weight_hsc), psf=scarlet2.ArrayPSF(jnp.asarray([psf_hsc0,])),channels=[channel_sc2],wcs=wcs_hsc)
        observations_sc2_old.append(obs_sc2)
        norms.append(AsinhPercentileNorm(data_hsc,percentiles=[1,95]))#norms.append(AsinhMapping(minimum=-1, stretch=5, Q=3))
epochs = np.asarray([obs.channels[0][0] for obs in observations])
bands0,inds0 = np.unique([obs.channels[0][0] for obs in observations],return_index=True)
bands = epochs[inds0]

model_psf = scarlet.GaussianPSF(sigma=0.7)
model_frame,offset = scarlet.Frame.from_observations(observations, coverage='intersection', model_psf=model_psf)
frame_psf_sc2 = scarlet2.GaussianPSF(0.7)
model_frame_sc2 = scarlet2.Frame(scarlet2.Box((len(observations_sc2_old),observations_sc2_old[0].data.shape[-1],observations_sc2_old[0].data.shape[-2])), psf=frame_psf_sc2, channels=channels_sc2,wcs=wcs_hsc)



Run source detection to create a catalog of source positions. Subtract background from images.

In [ ]:
lvl = 1
wave = 1
#Obtain source catalog and background flux estimation from our makeCatalog function
catalog_single, bgsingle, detectsingle, globalback = makeCatalog(observations, lvl, wave)
obssinglearr=np.asarray(observations)
bgsinglearr=np.asfarray(bgsingle)
pixel = np.stack((catalog_single['y'], catalog_single['x']), axis=1)
ra_dec = [obs.get_sky_coord(pixel) for obs in obssinglearr][0]
observations_sc2=[]
normsingle=[]
for ind,(obs,obs2,bg,back) in enumerate(zip(observations,observations_sc2_old,bgsingle,globalback)):
    #Reset weights based on background estimate (if needed)
    #obs.weights = np.ones(obs.shape) / (bg**2)[:, None, None]#(bg**2)[:, None, None]
    #Subtract background flux if needed
    #obs.data = obs.data-back
    #obs2.data = obs2.data - back
    obs_sc2 = scarlet2.Observation(jnp.asarray(obs.data), jnp.asarray(obs.weights), psf=scarlet2.ArrayPSF(jnp.asarray(obs.psf.get_model())),channels=[channels_sc2[ind]],wcs=wcs_hsc)
    observations_sc2.append(obs_sc2)
    #Store norm based on observation data
    normsingle.append(LinearPercentileNorm(obs.data,percentiles=[0.1, 99.9]))
    
for obs in observations_sc2:
    obs.match(model_frame_sc2)

Plot the first couple of images

In [ ]:
print(np.asarray(ra_dec).shape)
if np.asarray(ra_dec).shape==(2,):
    for ind,(obs, norm) in enumerate(zip(observations[:1], normsingle[:1])):
        scarlet.display.show_observation(obs, norm=norm, sky_coords=[ra_dec], show_psf=True)
else:    
    for ind,(obs, norm) in enumerate(zip(observations[:1], normsingle[:1])):
        scarlet.display.show_observation(obs, norm=norm, sky_coords=ra_dec, show_psf=True)
    

In [ ]:
if np.asarray(ra_dec).shape==(2,):
    separ=[0]
    indtransient=0
else:
    c2 = [SkyCoord(ra*u.deg,dec*u.deg,frame='icrs') for ra,dec in ra_dec]
    separ = [coord_transient.separation(c).arcsecond for c in c2]
    indtransient = np.argmin(separ)

In [ ]:
epochs = np.asarray([obs.channels[0][0] for obs in observations])
bands0,inds0 = np.unique([obs.channels[0][0] for obs in observations],return_index=True)
bands = epochs[inds0]
bandind = [np.argwhere(bands==e)[0][0] for e in epochs]

Define the scarlet2 PSF model and model frame. Match the model frame to the observations.

In [ ]:
frame_psf_sc2 = scarlet2.GaussianPSF(0.7)
model_frame_sc2 = scarlet2.Frame(scarlet2.Box((len(observations_sc2),observations_sc2[0].data.shape[-2],observations_sc2[0].data.shape[-1])), psf=frame_psf_sc2, channels=channels_sc2,wcs=wcs_hsc)

for obs in observations_sc2:
    obs.match(model_frame_sc2)

Initialize scarlet2 sources 

In [ ]:
from galaxygrad import HSC_ScoreNet64, ZTF_ScoreNet64

stepnum=600
gal_step = 2e-3
AGN_step = 2e-3
from scarlet2 import nn
model_size = 64

prior = nn.ScorePrior(
            model=HSC_ScoreNet64
        )
with scarlet2.Scene(model_frame_sc2) as scene:
    for i,pos in enumerate(ra_dec):      
        coord = SkyCoord(pos[0],pos[1],unit="deg")
        pospix = np.asfarray(observations_sc2_old[0].frame.wcs.world_to_pixel(coord))
        center = jnp.asarray(np.asfarray([pos[0],pos[1]]))
        centerpix = jnp.asarray([pospix[1],pospix[0]])

        if i==indtransient:    
            flux = initialization.pixel_spectrum(observations_sc2, center)
            flux = jnp.asarray(np.asarray(flux)[:,0])        
            n_steps, peak_value2 = stepnum, jnp.max(flux[~np.isinf(flux)])

            schedule3 = optax.cosine_onecycle_schedule(n_steps, peak_value2, final_div_factor=1)
            AGN_step = lambda p: scarlet2.relative_step(p, factor=5e-4)
            test_step_agn =1e-3*peak_value2
            
            scarlet2.PointSource(
                    centerpix,
                    scarlet2.TransientArraySpectrum(jnp.asarray(flux),epochs = channels_zeroed))#, stepsize=AGN_step, constraint=constraints.positive), epochs = channels_zeroed)#,constraint = scarlet2.PositiveConstraint())
                
            if ra_dec.shape[0]==1:
                flux = initialization.pixel_spectrum(observations_sc2, center)
                flux = jnp.asarray(np.asarray(flux)[:,0][inds0]) 
                morph_init = initialization.adaptive_gaussian_morph(observations_sc2[0], center) + 1e-6
                print('Making host at position of transient',flux,np.min(morph_init))
                morph_init[morph_init<1e-12] = 1e-12
                n_steps, peak_value = stepnum, 1

                schedule = optax.cosine_onecycle_schedule(n_steps, peak_value, final_div_factor=1)
                morph_step = lambda p: scarlet2.relative_step(p, factor=1e-3)#lambda p, it: scarlet2.relative_step(p, factor=2e-3) * schedule(it)
                
                test_step_morph =2e-3*peak_value
                n_steps, peak_value2 = stepnum, jnp.max(flux[~np.isinf(flux)])


                schedule2 = optax.cosine_onecycle_schedule(n_steps, peak_value2, final_div_factor=1)
                gal_step = lambda p: scarlet2.relative_step(p, factor=5e-4)#lambda p, it: scarlet2.relative_step(p, factor=2e-3) * schedule(it)
                test_step_gal =5e-3*peak_value2
                scarlet2.Source(
                    center,
                    scarlet2.StaticArraySpectrum(jnp.asarray(flux),filters=bandall),
                    scarlet2.ArrayMorphology(morph_init))
                   
        else:
            
            flux = jnp.asarray(np.asarray(initialization.pixel_spectrum(observations_sc2, center))[:,0][inds0])
            morph_init = initialization.adaptive_gaussian_morph(observations_sc2[0], center) + 1e-12
            print('Making other galaxy',flux,np.sum(np.isnan(morph_init)))
           
            morph_init = jnp.asarray(morph_init)
            n_steps, peak_value = stepnum, 1
           
            schedule = optax.cosine_onecycle_schedule(n_steps, peak_value, final_div_factor=1)
            morph_step = lambda p: scarlet2.relative_step(p, factor=1e-3)
            test_step_morph =2e-3*peak_value
            n_steps, peak_value2 = stepnum, jnp.max(flux[~np.isinf(flux)])
            
    
            schedule2 = optax.cosine_onecycle_schedule(n_steps, peak_value2, final_div_factor=1)
            gal_step = lambda p: scarlet2.relative_step(p, factor=5e-4)
            test_step_gal =5e-3*peak_value2

            scarlet2.Source(
                centerpix,
                scarlet2.StaticArraySpectrum(jnp.asarray(flux),filters=bandall),
                scarlet2.ArrayMorphology(morph_init))
pos_step = lambda p: scarlet2.relative_step(p, factor=1e-3)
parameters = scene.make_parameters()
for i in range(len(scene.sources)):
    if i==indtransient:
        parameters += Parameter(scene.sources[i].spectrum.data, name=f"spectrum.{i}", constraint=constraints.positive, stepsize=AGN_step)
        parameters += Parameter(scene.sources[i].morphology.center, name=f"spectrum.{i}", constraint=constraints.positive, stepsize=pos_step)
 
    else:
        parameters += Parameter(scene.sources[i].spectrum.data, name=f"spectrum.{i}", constraint=constraints.positive, stepsize=gal_step)
        parameters += Parameter(scene.sources[i].morphology.data, name=f"morph.{i}", constraint=constraints.positive, stepsize=morph_step)
        
scene.set_spectra_to_match(observations_sc2,parameters)

In [ ]:
scarlet2.plot.scene(
    scene,
    observation=observations_sc2[10],
    norm=normsingle[10],
    channel_map=None,
    show_model=False,
    show_observed=True,
    show_rendered=True,
    show_residual=True,
    add_labels=True,
    add_boxes=True,
    linear=True,
)
plt.show()

In [ ]:
import cmasher as cmr
cmap = cmr.lilac
int_method='none'
fig, axes = plt.subplots(1, len( scene.sources ), figsize=(15,6),dpi=120)
for i, ax in enumerate(axes):
    if i==indtransient:
        y = scene.sources[i].morphology()
    else:
        y = scene.sources[i].morphology.data
    ax.imshow(y, cmap = cmap,interpolation=int_method,vmin = 0)#np.max([np.min(np.log(y)[np.log(y)>-15]),-15]))
    ax.set_title(f"source {i}", fontsize = 18)
    ax.invert_yaxis()
plt.suptitle('reconstructed sources (transient = '+str(indtransient)+')', fontsize=12)#,y=0.75)
plt.show()
plt.clf()

In [ ]:
n_steps, peak_value_pos = stepnum, 1
schedulepos = optax.cosine_onecycle_schedule(n_steps, peak_value_pos, final_div_factor=1)
pos_step = lambda p: scarlet2.relative_step(p, factor=1e-3)

scene_ = scene.fit(observations_sc2, parameters,max_iter=n_steps, e_rel=1e-8,schedule=schedulepos)
flux = scene_.sources[indtransient].spectrum.data


In [ ]:
scarlet2.plot.scene(
    scene_,
    observation=observations_sc2[0],
    norm=normsingle[0],
    channel_map=None,
    show_model=False,
    show_observed=True,
    show_rendered=True,
    show_residual=True,
    add_labels=True,
    add_boxes=True,
    #figsize=None,
    linear=True,
)
plt.show()

In [ ]:
from numpyro.infer.initialization import init_to_sample

parameters = scene_.make_parameters()
p = scene_.sources[indtransient].morphology.center
prior = dist.Normal(p, scale=1)
parameters += Parameter(p, name=f"morphology.center:"+str(indtransient), prior=prior)
p = scene_.sources[indtransient+1].morphology.data
prior = dist.Normal(p, scale=1)
parameters += Parameter(p, name=f"morphology.data:"+str(indtransient+1), prior=prior)
p = scene_.sources[indtransient].spectrum.data
prior = dist.Normal(p, scale=1)
parameters += Parameter(p, name=f"spectrum:"+str(indtransient), prior=prior)
p = scene_.sources[indtransient+1].spectrum.data
prior = dist.Normal(p, scale=1)
parameters += Parameter(p, name=f"spectrum:"+str(indtransient+1), prior=prior)

mcmc = scene_.sample(observations_sc2,parameters,num_warmup=200, num_samples=300, dense_mass=True, init_strategy=init_to_sample) 
#print(mcmc.get_samples())
q = mcmc.get_samples()['morphology.center:'+str(indtransient)] 
#q = mcmc.get_samples()['spectrum:'+str(indtransient)] 
#q = mcmc.get_samples()['morphology.data:'+str(indtransient+1)] 

q1 = np.percentile(q,[2.5,16,50,84,97.5],axis=0) 
print('q1',q1)
centerAGN = np.asfarray([q1[2,0],q1[2,1]])
errorAGN = np.asarray([np.abs(q1[4,0]-q1[0,0]),np.abs(q1[4,1]-q1[0,1])])
print(centerAGN,errorAGN)
corner.corner(mcmc).show() 
plt.savefig(plotdir+'/'+srcname+'_transientcenter_corner.png')
plt.show()
plt.clf()

complc = np.loadtxt('/scratch/gpfs/cw1074/HSCdata/multiressingleshift_priorimage/SNoff1.5_5_2.0_lc.txt',dtype=str)
q = mcmc.get_samples()['spectrum:'+str(indtransient)] 
q1 = np.percentile(q,[2.5,16,50,84,97.5],axis=0) 
fac = np.sum(scene_.sources[indtransient].morphology(),axis=(-2,-1)) 
fluxes = [] 
fluxerrors = [] 
for i,b in zip(range(q1.shape[1]),bandind): 
    print(i,'median = ',q1[2][i],'std = ',(q1[3][i]-q1[1][i])) 
    fluxes.append(q1[2][i]*fac) 
    fluxerrors.append(np.abs(q1[3][i]-q1[1][i])*fac)

fluxes = np.asfarray(fluxes)
fluxerrors = np.asfarray(fluxerrors)
times=np.asfarray(times)


fac = np.sum(scene_.sources[indtransient].morphology(),axis=(-2,-1))

for bind,(b,c) in enumerate(zip(bandall,['green','orange','red','purple','blue'])):
    indb = np.argwhere(np.asarray(lcbands)==b)[:,0]
    plt.errorbar(times[epochs==b]-57700+1550,fluxes[epochs==b],yerr = fluxerrors[epochs==b],linestyle='',label=b+'-scarlet',color=c,marker='.',ecolor='gray')#+fluxgal[epochs==b]),label=b,color=c,marker='.')
    plt.scatter(np.asfarray(complc[:,0])[indb]-57700+1550,np.asfarray(complc[:,2])[indb],label=b+'-sim',color=c,marker='x')

plt.xlabel('MJD-56150')
plt.xlim((0,100))
plt.ylabel('Flux')
plt.legend()
plt.savefig(plotdir+'/'+srcname+'_lc_flux_nonparametric.pdf')
plt.show()
plt.clf()
outlchead=['MJD','FLUX','FLUXERR','BAND']
outlc=np.column_stack((times,fluxes,fluxerrors,epochs))
outlc = np.vstack((outlchead,outlc))
np.savetxt(plotdir+'/lc.txt',np.asarray(outlc,dtype=str),fmt='%s')
sed=[]
sederr=[]
print(bandall)
for b,c in zip(bandall,['green','orange','red','purple','blue']):
    indb = np.argwhere(np.asarray(lcbands)==b)[:,0]
    plt.errorbar(times[epochs==b]-57700+1550,25-2.5*np.log10(fluxes[epochs==b]),yerr = 1.0857*np.abs(fluxerrors[epochs==b]/fluxes[epochs==b]),ecolor='gray',linestyle='',label=b+'-scarlet',color=c,marker='.')#+fluxgal[epochs==b]),label=b,color=c,marker='.')
    plt.scatter(np.asfarray(complc[:,0])[indb]-57700+1550,25-2.5*np.log10(np.asfarray(complc[:,2])[indb]),label=b+'-sim',color=c,marker='x')
    summed_diff+=np.sum(np.abs(flux[epochs==b]-np.asfarray(complc[:,2])[indb]))
print('summed diff',summed_diff)
plt.xlabel('MJD-56150')
plt.xlim((0,100))
plt.ylim((19.8,24))
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()
plt.legend()
plt.savefig(plotdir+'/'+srcname+'_lc_mag_nonparametric.pdf')
print('Saved',plotdir+'/'+srcname+'_lc_mag_nonparametric.pdf')
plt.show()